# Sea Surface Temperature Bias - NOAA OISSTv2

* This notebook plots the SST bias of a model simulation vs. climataology derived from NOAA OISSTv2
* Authors: Xinru Li and John Krasting

In [ ]:


# For testing and development purposes, enter a start year, end year, and
# an dora id number to analyze. The value of dora_id can also be a direct
# path to a /pp directory.

config = {
    "startyr": None,
    "endyr": None,
    "dora_id": "odiv-413",
}

In [ ]:
# Make sure this cell is active so that the workflow and Dora can update
# the config dictionary at runtime.

#from gfdlnb.tools.update_notebook_config import update_notebook_config
#config = update_notebook_config(config)

In [ ]:
print(str(config))

### Import Python Modules

In [ ]:
import doralite
import glob
import momlevel
import subprocess
import os
import datetime

import intake
import numpy as np
import xarray as xr
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')  # non-X windows backend
import cartopy.crs as ccrs

from matplotlib.colors import ListedColormap, BoundaryNorm

In [ ]:
# momgrid will use a directory of pre-computed weights that is used for calculating basic area-weighted statistics later
import momgrid
os.environ["MOMGRID_WEIGHTS_DIR"] = "/nbhome/John.Krasting/grid_weights"

### Definie Local Parameters

In [ ]:
# Define some local variables. These are taken from the doralite object
# or they can be defined locally

if config["dora_id"] is not None:
    experiment =  doralite.dora_metadata(config["dora_id"])
    pathPP = experiment["pathPP"]
    expName = experiment["expName"]
else:
    raise ValueError("Experiment must be defined")

# Define start and end years
try: 
    start = os.environ["startdate"]
except KeyError:
    start = config["startyr"]
start = int(start) if start is not None else 1
try:
    end = os.environ["enddate"]
except KeyError:
    end = config["endyr"]
end = int(end) if end is not None else 9999

### Determine What Files to Load

In [ ]:
# Determine what files are needed (leave this up to the developer for flexibility)
# This is an example of what someone might do:
frequency = 'mon'

try:
    #  ^..^
    # /o  o\   
    # oo--oo~~~
    cat = intake.open_esm_datastore(os.environ['CATALOG_FILE'])
except KeyError:
    print(f"ERROR: Could not load catalog file {os.environ['CATALOG_FILE']}")

tos_subset = cat.search(variable_id='tos', frequency=frequency)
filelist = sorted([path for path in tos_subset.df['path']])
areacello_subset = cat.search(variable_id='areacello', frequency=frequency)
staticfile = sorted([path for path in tos_subset.df['path']])
staticfile = staticfile[0] if staticfile else None


# WARNING: LEGACY WAY OF GRABBING FILES
#component = "ocean_monthly"
#static = f"{component}/{component}.static.nc"
#varname = "tos"

#chunk = "5yr"
#filelist = sorted(glob.glob(f"{pathPP}{component}/ts/**/{chunk}/{component}.*.{varname}.nc", recursive=True))

#def is_in_range(file,start,end):
#    start = 1 if start is None else start
#    end = 9999 if end is None else end
#    target = set(list(range(start,end+1)))
#    fname = os.path.basename(file)
#    times = fname.split(".")[1]
#    times = times.split("-")
#    times = [int(x[0:4]) for x in times]
#    candidate = set(list(range(times[0],times[1]+1)))
#    return len(candidate.intersection(target)) > 0

#filelist = [x for x in filelist if is_in_range(x,start,end)]
#staticfile = f"{pathPP}/{static}"

#_ = [print(x) for x in filelist]

### DMgetting Files

Dora cannot issue calls to dmget

In [ ]:
if not "DORA_EXECUTE" in os.environ.keys():
    print("Calling dmget on files ...")
    cmd = ["dmget"]+filelist+[staticfile]
    _ = subprocess.check_output(cmd)

### Load model data and grid

In [ ]:
ds = momgrid.Gridset(filelist, force_symmetric=True, return_corners=True)
model_type = ds.model
ds = ds.data.sel(time=slice(f"{str(start).zfill(4)}-01-01",f"{str(end).zfill(4)}-12-31"))

In [ ]:
ds

### Load observational data (if needed)

Note that Dora only mounts `/archive`, `/nbhome`, and `/home`

In [ ]:
model_type = 'om5'
if "om4" in model_type:
    dsobs = xr.open_dataset("/archive/John.Krasting/NOAA_OISST_v2_annual_mean_1993-2017_OM4.nc", use_cftime=True)
elif "om5" in model_type:
    dsobs = xr.open_dataset("/archive/John.Krasting/NOAA_OISST_v2_annual_mean_1993-2017_OM5.nc", use_cftime=True)
else:
    raise ValueError(f"Unable to load obs for model type: {model_type}")

### Define some helper function for the plots

In [ ]:
def gen_levs_and_cmap(start,end,delta,cmap="RdBu_r"):
    """Generates a difference colormap centered on white"""
    boundaries = np.arange(start,end,delta)
    levels = (boundaries[0:-1] + boundaries[1:]) / 2.
    base_cmap = plt.get_cmap(cmap)
    colors = base_cmap(np.linspace(0, 1, len(levels)))
    colors[[int(len(colors) / 2) - 1]] = [1, 1, 1, 1]
    colors[[int(len(colors) / 2)]] = [1, 1, 1, 1]
    cmap = ListedColormap(colors)
    norm = BoundaryNorm(boundaries, cmap.N, clip=True)
    return (cmap, norm, boundaries)

In [ ]:
def set_annotaions(ax):
    _ = ax.set_xticks([])
    _ = ax.set_yticks([])
    _ = ax.text(0.0,1.06, "SST Bias Relative to NOAA OISSTv2 (1993-2017)", weight="bold", fontsize=12, transform=ax.transAxes)
    _ = ax.text(0.0,1.02, expName, style="italic", fontsize=10, transform=ax.transAxes)
    _ = ax.text(1.0,1.05, str(starttime.values), ha="right", fontsize=8, transform=ax.transAxes)
    _ = ax.text(1.0,1.02, str(endtime.values), ha="right", fontsize=8, transform=ax.transAxes)

In [ ]:
def add_stats_box(ax, stats_str, x=0.015, y=0.8):
        # Adding the text box annotation
    props = dict(
        boxstyle="round,pad=0.3", edgecolor="black", linewidth=1.5, facecolor="white"
    )
    ax.text(
        x,
        y,
        stats_str,
        transform=ax.transAxes,
        fontsize=8,
        verticalalignment="top",
        bbox=props,
    )

In [ ]:
def calculate_stats(model,obs,areacello):
    diff = model - obs
    stats = {}
    stats["min"] = float(diff.min())
    stats["max"] = float(diff.max())
    stats = {**stats, **momgrid.xr_stats.xr_stats_2d(model,obs,ds.areacello,fmt="dict")}
    # Limit to 4 significant digits
    stats = {k:f"{v:.4g}" for k,v in stats.items()}
    # Stats string
    stats_str = str("\n").join([f"{k} = {v}" for k,v in stats.items()])
    return (stats, stats_str)

In [ ]:
def add_colorbar(fig, cb, boundaries):
    cbar_ax = fig.add_axes([0.16, 0.06, 0.7, 0.03])
    fig.colorbar(
        cb, cax=cbar_ax, orientation="horizontal", extend="both", ticks=boundaries[::4]
    )

In [ ]:
momlevel.util.annual_average(ds)

### Perform some calculations

In [ ]:
# Time-average the model data
model = ds.tos
starttime = model.time[0]
endtime = model.time[-1]
model = momlevel.util.annual_average(model).mean("time", keep_attrs=True).load()

# Obs data is already a climatology
obs = dsobs.tos

### Establish a Dictionary to Store Scalar Metrics

In [ ]:
stats_dict = {}
stats_dict["metadata"] = {
    "expName": str(expName),
    "created": datetime.datetime.now().isoformat(),
    "starttime": str(starttime.values),
    "endtime": str(endtime.values),
}
stats_dict["results"] = {}
stats_dict

In [ ]:
%matplotlib inline
# Setup plot
fig = plt.figure(figsize=(10,6))
ax = plt.subplot(1,1,1, facecolor="lightgray")

# Definie geolon and geolat for plotting (use corners!)
x = ds.geolon_c
y = ds.geolat_c

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-4.5,4.75,0.25)

# Run pcolormesh
cb = plt.pcolormesh(x,y,model-obs, cmap=cmap, norm=norm)

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(model, obs, ds.areacello)
add_stats_box(ax, stats_str)

# Add colorbar
add_colorbar(fig, cb, boundaries)

plt.show()

# Save stats
stats_dict["results"]["global"] = stats

### Arctic

In [ ]:
xrange = (-298,61)
yrange = (60.,91.)

_model = momgrid.geoslice.geoslice(model,x=xrange,y=yrange)
_obs = momgrid.geoslice.geoslice(obs,x=xrange,y=yrange)

xq = list((_model.xh.values - 0.5).astype(int))
yq = list((_model.yh.values - 0.5).astype(int))

xq = xq + [xq[-1] + 1]
yq = yq + [yq[-1] + 1]

x = ds.geolon_c.values[yq,:][:,xq]
y = ds.geolat_c.values[yq,:][:,xq]

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-2.0,2.125,0.125)

fig = plt.figure(figsize=(6,6))
ax = plt.subplot(1,1,1, facecolor="lightgray", projection=ccrs.NorthPolarStereo())
cb = ax.pcolormesh(x,y,_model-_obs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
ax.coastlines()

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(_model, _obs, _model.areacello)
add_stats_box(ax, stats_str, x=0.05, y=0.95)

# Add colorbar
add_colorbar(fig, cb, boundaries)

# Save stats
stats_dict["results"]["Arctic"] = stats

### Southern Ocean

In [ ]:
xrange = (-300,60)
yrange = (-60.,-91.)

_model = momgrid.geoslice.geoslice(model,x=xrange,y=yrange)
_obs = momgrid.geoslice.geoslice(obs,x=xrange,y=yrange)

xq = list((_model.xh.values - 0.5).astype(int))
yq = list((_model.yh.values - 0.5).astype(int))

xq = xq + [xq[-1] + 1]
yq = yq + [yq[-1] + 1]

x = ds.geolon_c.values[yq,:][:,xq]
y = ds.geolat_c.values[yq,:][:,xq]

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-2.0,2.125,0.125)

fig = plt.figure(figsize=(6,6))
ax = plt.subplot(1,1,1, facecolor="lightgray", projection=ccrs.SouthPolarStereo())
cb = ax.pcolormesh(x,y,_model-_obs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
ax.coastlines()

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(_model, _obs, _model.areacello)
add_stats_box(ax, stats_str, x=0.5, y=0.7)

# Add colorbar
add_colorbar(fig, cb, boundaries)

# Save stats
stats_dict["results"]["southern_ocean"] = stats

### Northwest Pacific

In [ ]:
xrange = (-250.,-150.)
yrange = (25.,60.)

_model = momgrid.geoslice.geoslice(model,x=xrange,y=yrange)
_obs = momgrid.geoslice.geoslice(obs,x=xrange,y=yrange)

xq = list((_model.xh.values - 0.5).astype(int))
yq = list((_model.yh.values - 0.5).astype(int))

xq = xq + [xq[-1] + 1]
yq = yq + [yq[-1] + 1]

x = ds.geolon_c.values[yq,:][:,xq]
y = ds.geolat_c.values[yq,:][:,xq]

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-4.5,4.75,0.25)

fig = plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1, facecolor="lightgray", projection=ccrs.Miller(central_longitude=-200))
cb = ax.pcolormesh(x,y,_model-_obs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
ax.coastlines()

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(_model, _obs, _model.areacello)
add_stats_box(ax, stats_str)

# Add colorbar
add_colorbar(fig, cb, boundaries)

# Save stats
stats_dict["results"]["Northest_Pacific"] = stats

### Tropical and Subtropical Indo-Pacific

In [ ]:
xrange = (-280.,-80.)
yrange = (-23.,23.)

_model = momgrid.geoslice.geoslice(model,x=xrange,y=yrange)
_obs = momgrid.geoslice.geoslice(obs,x=xrange,y=yrange)

xq = list((_model.xh.values - 0.5).astype(int))
yq = list((_model.yh.values - 0.5).astype(int))

xq = xq + [xq[-1] + 1]
yq = yq + [yq[-1] + 1]

x = ds.geolon_c.values[yq,:][:,xq]
y = ds.geolat_c.values[yq,:][:,xq]

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-4.5,4.75,0.25)

fig = plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1, facecolor="lightgray", projection=ccrs.Miller(central_longitude=-180))
cb = ax.pcolormesh(x,y,_model-_obs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
ax.coastlines()

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(_model, _obs, _model.areacello)
add_stats_box(ax, stats_str, x=0.005, y=0.98)

# Add colorbar
add_colorbar(fig, cb, boundaries)

# Save stats
stats_dict["results"]["TropicalSubtropical_IndoPacific"] = stats

### South Indo-Pacific

In [ ]:
xrange = (-300.,-170.)
yrange = (-25.,-60.)

_model = momgrid.geoslice.geoslice(model,x=xrange,y=yrange)
_obs = momgrid.geoslice.geoslice(obs,x=xrange,y=yrange)

xq = list((_model.xh.values - 0.5).astype(int))
yq = list((_model.yh.values - 0.5).astype(int))

xq = xq + [xq[-1] + 1]
yq = yq + [yq[-1] + 1]

x = ds.geolon_c.values[yq,:][:,xq]
y = ds.geolat_c.values[yq,:][:,xq]

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-4.5,4.75,0.25)

fig = plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1, facecolor="lightgray", projection=ccrs.Miller(central_longitude=-180))
cb = ax.pcolormesh(x,y,_model-_obs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
ax.coastlines()

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(_model, _obs, _model.areacello)
add_stats_box(ax, stats_str, x=0.005, y=0.98)

# Add colorbar
add_colorbar(fig, cb, boundaries)

# Save stats
stats_dict["results"]["south_indo_pacific"] = stats

### North Atlantic 

In [ ]:
xrange = (-80.,-0.)
yrange = (30.,60.)

_model = momgrid.geoslice.geoslice(model,x=xrange,y=yrange)
_obs = momgrid.geoslice.geoslice(obs,x=xrange,y=yrange)

xq = list((_model.xh.values - 0.5).astype(int))
yq = list((_model.yh.values - 0.5).astype(int))

xq = xq + [xq[-1] + 1]
yq = yq + [yq[-1] + 1]

x = ds.geolon_c.values[yq,:][:,xq]
y = ds.geolat_c.values[yq,:][:,xq]

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-4.5,4.75,0.25)

fig = plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1, facecolor="lightgray", projection=ccrs.Miller(central_longitude=-60))
cb = ax.pcolormesh(x,y,_model-_obs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
ax.coastlines()

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(_model, _obs, _model.areacello)
add_stats_box(ax, stats_str, x=0.05, y=0.9)

# Add colorbar
add_colorbar(fig, cb, boundaries)

# Save stats
stats_dict["results"]["North_Atlantic"] = stats

### South Atlantic

In [ ]:
xrange = (-75.,25.)
yrange = (-20.,-60.)

_model = momgrid.geoslice.geoslice(model,x=xrange,y=yrange)
_obs = momgrid.geoslice.geoslice(obs,x=xrange,y=yrange)

xq = list((_model.xh.values - 0.5).astype(int))
yq = list((_model.yh.values - 0.5).astype(int))

xq = xq + [xq[-1] + 1]
yq = yq + [yq[-1] + 1]

x = ds.geolon_c.values[yq,:][:,xq]
y = ds.geolat_c.values[yq,:][:,xq]

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-4.5,4.75,0.25)

fig = plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1, facecolor="lightgray", projection=ccrs.Miller(central_longitude=-25))
cb = ax.pcolormesh(x,y,_model-_obs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
ax.coastlines()

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(_model, _obs, _model.areacello)
add_stats_box(ax, stats_str, x=0.05, y=0.98)

# Add colorbar
add_colorbar(fig, cb, boundaries)

# Save stats
stats_dict["results"]["South_Atlantic"] = stats

### California Current

In [ ]:
xrange = (-130.,-100.)
yrange = (18.,46.)

_model = momgrid.geoslice.geoslice(model,x=xrange,y=yrange)
_obs = momgrid.geoslice.geoslice(obs,x=xrange,y=yrange)

xq = list((_model.xh.values - 0.5).astype(int))
yq = list((_model.yh.values - 0.5).astype(int))

xq = xq + [xq[-1] + 1]
yq = yq + [yq[-1] + 1]

x = ds.geolon_c.values[yq,:][:,xq]
y = ds.geolat_c.values[yq,:][:,xq]

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-4.5,4.75,0.25)

fig = plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1, facecolor="lightgray", projection=ccrs.Miller(central_longitude=-115))
cb = ax.pcolormesh(x,y,_model-_obs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
ax.coastlines()

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(_model, _obs, _model.areacello)
add_stats_box(ax, stats_str, x=0.5, y=0.9)

# Add colorbar
add_colorbar(fig, cb, boundaries)

# Save stats
stats_dict["results"]["California_Current"] = stats

### Bengula Current

In [ ]:
xrange = (-10.,20.)
yrange = (35.,-30.)

_model = momgrid.geoslice.geoslice(model,x=xrange,y=yrange)
_obs = momgrid.geoslice.geoslice(obs,x=xrange,y=yrange)

xq = list((_model.xh.values - 0.5).astype(int))
yq = list((_model.yh.values - 0.5).astype(int))

xq = xq + [xq[-1] + 1]
yq = yq + [yq[-1] + 1]

x = ds.geolon_c.values[yq,:][:,xq]
y = ds.geolat_c.values[yq,:][:,xq]

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-4.5,4.75,0.25)

fig = plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1, facecolor="lightgray", projection=ccrs.Miller(central_longitude=5))
cb = ax.pcolormesh(x,y,_model-_obs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
ax.coastlines()

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(_model, _obs, _model.areacello)
add_stats_box(ax, stats_str, x=0.2, y=0.9)

# Add colorbar
add_colorbar(fig, cb, boundaries)

# Save stats
stats_dict["results"]["Bengula_Current"] = stats

### Off the west coast of South America

In [ ]:
xrange = (-110.,-60.)
yrange = (0.,-40.)

_model = momgrid.geoslice.geoslice(model,x=xrange,y=yrange)
_obs = momgrid.geoslice.geoslice(obs,x=xrange,y=yrange)

xq = list((_model.xh.values - 0.5).astype(int))
yq = list((_model.yh.values - 0.5).astype(int))

xq = xq + [xq[-1] + 1]
yq = yq + [yq[-1] + 1]

x = ds.geolon_c.values[yq,:][:,xq]
y = ds.geolat_c.values[yq,:][:,xq]

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-4.5,4.75,0.25)

fig = plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1, facecolor="lightgray", projection=ccrs.Miller(central_longitude=-85))
cb = ax.pcolormesh(x,y,_model-_obs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
ax.coastlines()

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(_model, _obs, _model.areacello)
add_stats_box(ax, stats_str, x=0.7, y=0.98)

# Add colorbar
add_colorbar(fig, cb, boundaries)

# Save stats
stats_dict["results"]["offthe_WestCoastSouthAmerica"] = stats

### Carribbean region

In [ ]:
xrange = (-90.,-60.)
yrange = (5.,30.)

_model = momgrid.geoslice.geoslice(model,x=xrange,y=yrange)
_obs = momgrid.geoslice.geoslice(obs,x=xrange,y=yrange)

xq = list((_model.xh.values - 0.5).astype(int))
yq = list((_model.yh.values - 0.5).astype(int))

xq = xq + [xq[-1] + 1]
yq = yq + [yq[-1] + 1]

x = ds.geolon_c.values[yq,:][:,xq]
y = ds.geolat_c.values[yq,:][:,xq]

# Get colormap
cmap, norm, boundaries = gen_levs_and_cmap(-4.5,4.75,0.25)

fig = plt.figure(figsize=(10,4))
ax = plt.subplot(1,1,1, facecolor="lightgray", projection=ccrs.Miller(central_longitude=-60))
cb = ax.pcolormesh(x,y,_model-_obs,transform=ccrs.PlateCarree(), cmap=cmap, norm=norm)
ax.coastlines()

# Clean up figure and add labels
set_annotaions(ax)

# Add statistics
stats, stats_str = calculate_stats(_model, _obs, _model.areacello)
add_stats_box(ax, stats_str, x=0.65, y=0.21)

# Add colorbar
add_colorbar(fig, cb, boundaries)

# Save stats
stats_dict["results"]["Carribbean"] = stats

### Write Stats to a File

In [ ]:
if not "DORA_EXECUTE" in os.environ.keys():
    import yaml
    yml = yaml.dump(stats_dict)
    with open('oisst_stats.yaml', 'w') as file:
        file.write(yml)
    file.close()
    print(yml)